# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,30.63,80,97,4.45,US,1706574708
1,1,st. john's,47.5649,-52.7093,25.97,94,100,33.38,CA,1706574463
2,2,santa luzia,-19.7697,-43.8514,79.70,80,0,3.44,BR,1706574708
3,3,ancud,-41.8697,-73.8203,56.53,75,100,2.06,CL,1706574708
4,4,port-aux-francais,-49.3500,70.2167,38.95,71,99,37.47,TF,1706574708


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

# Configure the map plot

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.6
)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 25 degrees
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 25)]
                                
#Wind speed less than 10
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 10)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] < 15)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,petropavlovsk-kamchatsky,53.0452,158.6483,3.79,78,0,6.71,RU,1706574639
122,122,happy valley-goose bay,53.3168,-60.3315,-12.69,53,0,5.75,CA,1706574729
128,128,chibougamau,49.9168,-74.3659,2.82,66,0,8.05,CA,1706574730
235,235,hovd,48.0056,91.6419,-7.22,86,10,1.43,MN,1706574751
312,312,maloye ganyushkino,46.6000,49.2667,18.34,77,10,8.28,KZ,1706574764
353,353,ust'-ilimsk,58.0006,102.6619,-34.56,100,7,1.01,RU,1706574771
424,424,shitanjing,39.2342,106.3439,23.70,29,0,3.56,CN,1706574785
467,467,lobujya,27.9500,86.8167,-3.24,47,0,3.94,NP,1706574795
495,495,corner brook,48.9667,-57.9484,1.56,96,6,3.04,CA,1706574801
510,510,zhangye,38.9342,100.4517,24.42,26,7,0.94,CN,1706574803


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

# Corrected code using a list instead of a dictionary
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Adding a new column to the hotel_df
hotel_df["Hotel Name"] = ""

# Displaying the resulting DataFrame
print(hotel_df)

                         City Country      Lat       Lng  Humidity Hotel Name
50   petropavlovsk-kamchatsky      RU  53.0452  158.6483        78           
122    happy valley-goose bay      CA  53.3168  -60.3315        53           
128               chibougamau      CA  49.9168  -74.3659        66           
235                      hovd      MN  48.0056   91.6419        86           
312        maloye ganyushkino      KZ  46.6000   49.2667        77           
353               ust'-ilimsk      RU  58.0006  102.6619       100           
424                shitanjing      CN  39.2342  106.3439        29           
467                   lobujya      NP  27.9500   86.8167        47           
495              corner brook      CA  48.9667  -57.9484        96           
510                   zhangye      CN  38.9342  100.4517        26           


C:\Users\Amanda\AppData\Local\Temp\ipykernel_20160\3450456209.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting search
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
happy valley-goose bay - nearest hotel: Hotel North Two
chibougamau - nearest hotel: No hotel found
hovd - nearest hotel: Khovd hotel
maloye ganyushkino - nearest hotel: Конил Сазы
ust'-ilimsk - nearest hotel: Усть-Илимск
shitanjing - nearest hotel: No hotel found
lobujya - nearest hotel: Himalaya Lodge
corner brook - nearest hotel: No hotel found
zhangye - nearest hotel: Zhangye Hotel 张掖饭店


,City,Country,Lat,Lng,Humidity,Hotel Name
50,petropavlovsk-kamchatsky,RU,53.0452,158.6483,78,ООО Постоялый двор
122,happy valley-goose bay,CA,53.3168,-60.3315,53,Hotel North Two
128,chibougamau,CA,49.9168,-74.3659,66,No hotel found
235,hovd,MN,48.0056,91.6419,86,Khovd hotel
312,maloye ganyushkino,KZ,46.6000,49.2667,77,Конил Сазы
353,ust'-ilimsk,RU,58.0006,102.6619,100,Усть-Илимск
424,shitanjing,CN,39.2342,106.3439,29,No hotel found
467,lobujya,NP,27.9500,86.8167,47,Himalaya Lodge
495,corner brook,CA,48.9667,-57.9484,96,No hotel found
510,zhangye,CN,38.9342,100.4517,26,Zhangye Hotel 张掖饭店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 800,
    size = "Humidity",
    color="City",
    alpha=.4,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)